# プロダクトマニュアルをVector Search Indexで検索可能な状態にする(標準時間:30分)

## 本ノートブックの目的：Databricks における Vector Search のテキスト検索と Index へのガバナンスを理解する
Q1. プロダクトマニュアルの前処理<br>
Q2. Vector Search Index の作成<br>
Q3. Vector Search のツール化とアクセス制御

## 事前準備 (標準時間：10分)

In [0]:
%pip install -U bs4 langchain_core langchain_text_splitters databricks-vectorsearch pydantic

In [0]:
%restart_python

In [0]:
%run ./00_config

In [0]:
# 本ノートブックで利用するスキーマを作成
schema_name = f"05_vector_search_index_for_{user_name}"
print(f"schema_name: `{schema_name}`")
spark.sql(
    f"""
    CREATE SCHEMA IF NOT EXISTS {catalog_name}.{schema_name}
    """
)
spark.sql(f"USE {catalog_name}.{schema_name}")

# 本ノートブックで利用するボリュームを作成
volume_name = f"product_manual"
print(f"volume_name: `{volume_name}`")
spark.sql(
    f"""
    CREATE VOLUME IF NOT EXISTS {catalog_name}.{schema_name}.{volume_name}
    """
)

### ソースデータファイルのダウンロードとボリュームへのアップロード
1. 現在のノートブックの左型タブにある`Workspace (Ctrl + Alt + E)`を選択し、現在のディレクトリにある`data`フォルダに移動
1. `product_manual`フォルダのハンバーガーメニュー（`︙`）を選択して、 `ダウンロード形式：` -> `ソースファイル` をクリックしてデータファイルをダウンロード
1. ダウンロードした Zip ファイルを解凍
1. 現在のノートブックの左型タブにある`Catalog (Ctrl + Alt + C)`を選択後、`05_vector_search_index_for_{username}`スキーマ下にある`product_manual` Volume にてハンバーガーメニュー（`︙`）を選択し、`このボリュームにアップロード`を選択
1. 表示されたウィンドウに解凍した CSV ファイルをすべて配置して、`アップロード`を選択
1. 下記のセルを実行し、ファイルが配置されていることを確認

In [0]:
# ファイルが配置されていることを確認
src_file_dir = f"/Volumes/{catalog_name}/{schema_name}/{volume_name}"
file_list = dbutils.fs.ls(src_file_dir)
if file_list == []:
    raise Exception("ファイルがディレクトリにありません。ファイルを配置してください。")
display(dbutils.fs.ls(src_file_dir))

## Q1. プロダクトマニュアルの前処理(標準時間：10分)
ダウンロード後に解凍した`product_manual`フォルダから、`index.html`を開いてみましょう。 \
マニュアルはHTMLで記述され、FAQやトラブルシューティングといったセクションに分かれています。 

起票されたサポートチケットに対してマニュアルに基づいて適切な回答を作成するためには、マニュアルの記載をLLMに入力して適切な回答を考えてもらう必要があります。 \
しかし、膨大なマニュアルの全てをLLMへ入力することはモデルでは通常入力可能なトークンの数が決まっているため不可能であり、コストや性能の観点からも望ましくありません。

このため、マニュアルを小さな塊に分割し、クエリと関連しそうな記述の塊（一般的にチャンクと呼ばれます）を抽出して、LLMへ与える文脈情報とします。今回はHTMLのH2タグによってHTMLファイルを分割します。

### 実践例(langchainの公式ドキュメントから引用)
HTMLの構造は以下のようにタグとその中のコンテンツを区別して簡潔に説明できます：

1. h1タグ: 最も重要な大見出し
   - ページの主題やタイトルを表す
   - 通常、1ページに1つのみ使用

2. h2タグ: 中見出し
   - 主要なセクションや章を表す
   - h1の下位階層として複数使用可能

3. h3タグ: 小見出し
   - h2の下位階層として使用
   - セクション内の重要なポイントを表す

4. pタグ: 段落
   - 本文のテキストを含む
   - 見出しの下に配置され、関連するコンテンツを表示

今回はBeautifulSoupと呼ばれるHTMLの解析を行うPythonパッケージを用いて、マニュアルの各ファイルを分割し、テーブル形式で保存します。

In [0]:
html_string = """
<!DOCTYPE html>
<html lang='ja'>
<head>
  <meta charset='UTF-8'>
  <meta name='viewport' content='width=device-width, initial-scale=1.0'>
  <title>おしゃれなHTMLページの例</title>
</head>
<body>
  <h1>メインタイトル</h1>
  <p>これは基本的な内容を含む導入段落です。</p>
  
  <h2>セクション1：はじめに</h2>
  <p>このセクションではトピックを紹介します。以下はリストです：</p>
  <ul>
    <li>最初の項目</li>
    <li>2番目の項目</li>
    <li><strong>太字のテキスト</strong>と<a href='#'>リンク</a>を含む3番目の項目</li>
  </ul>
  
  <h3>サブセクション1.1：詳細</h3>
  <p>このサブセクションでは追加の詳細を提供します。以下は表です：</p>
  <table border='1'>
    <thead>
      <tr>
        <th>ヘッダー1</th>
        <th>ヘッダー2</th>
        <th>ヘッダー3</th>
      </tr>
    </thead>
    <tbody>
      <tr>
        <td>1行目、1列目</td>
        <td>1行目、2列目</td>
        <td>1行目、3列目</td>
      </tr>
      <tr>
        <td>2行目、1列目</td>
        <td>2行目、2列目</td>
        <td>2行目、3列目</td>
      </tr>
    </tbody>
  </table>

  <h2>結論</h2>
  <p>これは文書の結論です。</p>
</body>
</html>


"""

In [0]:
from bs4 import BeautifulSoup

def split_html_without_headers(html_string):
    # h1, h2, h3, pタグのヘッダーを抽出する
    soup = BeautifulSoup(html_string, 'html.parser')
    headers = soup.find_all(['h1', 'h2', 'h3'])
    
    # h1, h2, h3, pタグのヘッダーを除いたコンテンツ
    chunks = []
    for header in headers:
        content = ''
        for sibling in header.find_next_siblings():
            if sibling.name in ['h1', 'h2', 'h3']:
                break
            content += sibling.get_text()
        
        if content.strip():
            chunks.append(content.strip())
    
    return chunks

display(
    split_html_without_headers(html_string)
    )

## ToDo: HTMLファイルのレコードをチャンクへ分割し、`product_documentation`テーブルへ格納

In [0]:
import pyspark.sql.functions as F

html_file = (
    spark
    .read
    .option("wholetext", True)
    .text(f"/Volumes/{catalog_name}/{schema_name}/{volume_name}/*.html")
    .withColumn("filename", F.col("_metadata.file_path"))
    .withColumn("filename", F.regexp_replace(F.col("filename"), r'^.*/', ''))
    )

display(html_file)

In [0]:
import pandas as pd

@F.pandas_udf("array<string>")
def parse_and_split(docs: pd.Series) -> pd.Series:
    return docs.apply(split_html_without_headers)

In [0]:
(
    html_file
    .withColumn("content", F.explode(parse_and_split("value")))
    .write.mode('overwrite').saveAsTable("product_documentation")
)

In [0]:
%sql
SELECT * FROM product_documentation

## Q2. Vector Search Index の作成(標準時間：10分)

### ベクトル検索とは
![ベクトル検索とは](/Workspace/Users/shotaro.kotani@nssoldlp.onmicrosoft.com/openhack2025/contents_with_answer/dev_shotkotani/data/img/vector_index.png)

この画像は、ベクトル検索の概念をわかりやすく表現しています。

*   **ベクトル表現（赤い点）:** 各赤い点は、特定のデータオブジェクト（例えば、画像、テキストドキュメント）のベクトル表現を表しています。ディープラーニングモデルなどの手法を用いて、データオブジェクトはその意味や内容に基づいて数値ベクトルに変換されます。
*   **クラスタリング:** 同様のデータオブジェクトは、多次元空間内で互いに近い位置に集まり、クラスタを形成します。画像の例では、「Dogs」と「Cat」のアイコンで示されたクラスタが、犬と猫の画像に対応しています。
*   **クエリベクトル（青い点）:** 「find me all cats」というテキストのクエリは、同様にベクトルに変換され、多次元空間内の青い点で表されます。
*   **類似度検索:** ベクトル検索では、クエリベクトルに最も近いデータベクトルを見つけます。画像では、青い点から猫のクラスタ内の赤い点への線が、この類似度を示しています。

要するに、ベクトル検索は、データオブジェクトをその意味や内容に基づいて多次元空間に配置し、クエリとデータオブジェクト間の距離（類似度）を計算することで、関連性の高い情報を効率的に見つけ出す技術です。


In [0]:
import mlflow.deployments

# Databricks のデプロイメントクライアントを取得
deploy_client = mlflow.deployments.get_deploy_client("databricks")

# OpenAI の text-embedding-ada-002 を使用（エンベディング生成）を実行
# エンドポイント "openhack-text-embedding-ada-002" に対して、"What is Apache Spark?" という入力テキストを渡す
response = deploy_client.predict(
    endpoint="openhack-text-embedding-ada-002",
    inputs={"input": ["Apache Spark とはなんでしょう?"]}
)

# 結果を取得
embeddings = [e["embedding"] for e in response.data]
print(embeddings)

### produc

In [0]:
# 以下はVector Searchエンドポイントの作成例です。
# Vector Searchエンドポイントの作成には10分～15分ほどかかります。 本日は時間がないので、作成済みのエンドポイントを利用します。

from databricks.vector_search.client import VectorSearchClient
vsc = VectorSearchClient()

if not endpoint_exists(vsc, VECTOR_SEARCH_ENDPOINT_NAME):
    vsc.create_endpoint(name=VECTOR_SEARCH_ENDPOINT_NAME, endpoint_type="STANDARD")

wait_for_vs_endpoint_to_be_ready(vsc, VECTOR_SEARCH_ENDPOINT_NAME)
print(f"Endpoint named {VECTOR_SEARCH_ENDPOINT_NAME} is ready.")

